# Processo Seletivo- Casa dos Ventos - Analytics - Case
## Candidato: Érick Tenorio da Silva
### Extração, processamento e armazenamento dos dados de aerogeradores para análise

### Data Extract

In [22]:
import requests
url = "https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query"
params = {
        'where': '1=1',
        'outFields': '*',
        'f': 'geojson'  
    }
res = requests.get(url,params=params)
data = res.json()

### Data Processing

Transformação em geodataframe

In [23]:
import geopandas as gpd
gdf = gpd.GeoDataFrame.from_features(data['features'])
gdf['Latitude'] = gdf.geometry.apply(lambda geom: geom.y if geom else None)
gdf['Longitude'] = gdf.geometry.apply(lambda geom: geom.x if geom else None)

Exploração e limpeza inicial

In [24]:
gdf.head()

,geometry,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,...,DATUM_EMP,OPERACAO,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,Latitude,Longitude
0,POINT (-42.64249 -14.47097),1.85,121.25,80.0,82.5,1666625847000,30739,Caititu,CAI-083,754125.00,...,WGS84,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,None,1,BA,EOLCVBA030739-4-01,-14.470968,-42.642495
1,POINT (-35.92447 -5.11575),2.00,145.00,95.0,100.0,1666625847000,30512,Dreen Boa Vista,BV07,175728.00,...,SIRGAS2000,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,None,2,RN,EOLCVRN030512-0-01,-5.115754,-35.924469
2,POINT (-35.84437 -5.26742),2.00,145.00,95.0,100.0,1666625847000,30497,Renascença V,15,184692.00,...,SIRGAS2000,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,None,3,RN,EOLCVRN030497-2-01,-5.267424,-35.844373
3,POINT (-35.92329 -5.44208),2.70,150.00,89.0,122.0,1666625847000,30864,SM,1,176028.00,...,SIRGAS2000,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,None,4,RN,EOLCVRN030864-1-01,-5.442082,-35.923294
4,POINT (-38.87983 -3.54033),2.10,143.70,96.2,95.0,1666625847000,30295,Colônia,AEG-08,513347.87,...,SIRGAS2000,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,None,5,CE,EOLCVCE030295-3-01,-3.540330,-38.879826


In [25]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   geometry          1000 non-null   geometry
 1   POT_MW            1000 non-null   float64 
 2   ALT_TOTAL         1000 non-null   float64 
 3   ALT_TORRE         1000 non-null   float64 
 4   DIAM_ROTOR        1000 non-null   float64 
 5   DATA_ATUALIZACAO  1000 non-null   int64   
 6   EOL_VERSAO_ID     1000 non-null   int64   
 7   NOME_EOL          1000 non-null   object  
 8   DEN_AEG           1000 non-null   object  
 9   X                 1000 non-null   float64 
 10  Y                 1000 non-null   float64 
 11  VERSAO            1000 non-null   object  
 12  DATUM_EMP         1000 non-null   object  
 13  OPERACAO          998 non-null    object  
 14  FUSO_AG           1000 non-null   object  
 15  PROPRIETARIO      1000 non-null   object  
 16  ORIGEM           

In [26]:
gdf['VERSAO'].unique() # Coluna irrelevante, pois há apenas um tipo de string

array(['Versão Válida'], dtype=object)

Transformação da coluna de datas

In [27]:
from datetime import datetime
gdf['Data_Atualizacao'] = gdf.DATA_ATUALIZACAO.apply(lambda data_int:datetime.fromtimestamp(data_int/1000.0) )

Adicionar uma coluna para região do Brasil

In [28]:
from funcoes import get_regiao
gdf['Regiao'] = gdf['UF'].apply(get_regiao)

Adicionar coluna para município

In [ ]:
# Arquivo de dados geográficos dos municípios brasileiros disponível em 
# https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html
file_path = 'C:\\Users\\Érick Tenorio\\Desktop\\ITA\\BR_Municipios_2022\\BR_Municipios_2022.shp'
municipios_gdf = gpd.read_file(file_path)
# Fazer a junção espacial para encontrar o município correspondente a cada ponto do Aerogerador
gdf = gpd.sjoin(gdf, municipios_gdf, how='left', op='within') # Adiciona outras colunas irrelevantes que serão eliminadas depois 
# (CD_MUN,SIGLA_UF,index_right e AREA_KM2)


Eliminar colunas com informações irrelevantes, arbitrárias ou com valores nulos.

In [30]:
gdf.drop(["geometry","ORIGEM","DATA_ATUALIZACAO","VERSAO","CD_MUN","SIGLA_UF","AREA_KM2","index_right"],axis=1,inplace=True)

Amostra final do dataset a ser salvo

In [31]:
gdf.head()

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,DATUM_EMP,...,FUSO_AG,PROPRIETARIO,OBJECTID,UF,CEG,Latitude,Longitude,Data_Atualizacao,Regiao,NM_MUN
0,1.85,121.25,80.0,82.5,30739,Caititu,CAI-083,754125.00,8398879.00,WGS84,...,UTM 23 S (MER -45),Central Eólica Caititu S.A.,1,BA,EOLCVBA030739-4-01,-14.470968,-42.642495,2022-10-24 12:37:27,Nordeste,Pindaí
1,2.00,145.00,95.0,100.0,30512,Dreen Boa Vista,BV07,175728.00,9433802.00,SIRGAS2000,...,UTM 25 S (MER -33),GE Boa Vista S.A.,2,RN,EOLCVRN030512-0-01,-5.115754,-35.924469,2022-10-24 12:37:27,Nordeste,São Bento do Norte
2,2.00,145.00,95.0,100.0,30497,Renascença V,15,184692.00,9417056.00,SIRGAS2000,...,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,3,RN,EOLCVRN030497-2-01,-5.267424,-35.844373,2022-10-24 12:37:27,Nordeste,Parazinho
3,2.70,150.00,89.0,122.0,30864,SM,1,176028.00,9397684.00,SIRGAS2000,...,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,4,RN,EOLCVRN030864-1-01,-5.442082,-35.923294,2022-10-24 12:37:27,Nordeste,João Câmara
4,2.10,143.70,96.2,95.0,30295,Colônia,AEG-08,513347.87,9608681.26,SIRGAS2000,...,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,5,CE,EOLCVCE030295-3-01,-3.540330,-38.879826,2022-10-24 12:37:27,Nordeste,São Gonçalo do Amarante


In [32]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   POT_MW            1000 non-null   float64       
 1   ALT_TOTAL         1000 non-null   float64       
 2   ALT_TORRE         1000 non-null   float64       
 3   DIAM_ROTOR        1000 non-null   float64       
 4   EOL_VERSAO_ID     1000 non-null   int64         
 5   NOME_EOL          1000 non-null   object        
 6   DEN_AEG           1000 non-null   object        
 7   X                 1000 non-null   float64       
 8   Y                 1000 non-null   float64       
 9   DATUM_EMP         1000 non-null   object        
 10  OPERACAO          998 non-null    object        
 11  FUSO_AG           1000 non-null   object        
 12  PROPRIETARIO      1000 non-null   object        
 13  OBJECTID          1000 non-null   int64         
 14  UF               

Identificação dos atributos a serem salvos:

- POT_MW ( type: esriFieldTypeDouble, alias: Potência (MW) )
- ALT_TOTAL ( type: esriFieldTypeDouble, alias: Altura total )
- ALT_TORRE ( type: esriFieldTypeDouble, alias: Altura da torre )
- DIAM_ROTOR ( type: esriFieldTypeDouble, alias: Diâmetro do rotor )
- Data_Atualizacao ( type: esriFieldTypeDatetime64, alias: Data da atualização, length: 8 )
- EOL_VERSAO_ID ( type: esriFieldTypeInteger, alias: ID Empreendimento )
- NOME_EOL ( type: esriFieldTypeString, alias: Nome da EOL, length: 254 )
- DEN_AEG ( type: esriFieldTypeString, alias: Denominação do AG, length: 100 )
- X ( type: esriFieldTypeDouble, alias: E )
- Y ( type: esriFieldTypeDouble, alias: N )
- VERSAO ( type: esriFieldTypeString, alias: Versão atual,desc:Indica a versão do empreendimento,  length: 50 )
- OPERACAO ( type: esriFieldTypeString, alias: Operação Comercial, length: 50 , Coded Values: [1: SIM] , [2: NÃO] )
- FUSO_AG ( type: esriFieldTypeString, alias: Fuso, length: 50 )
- PROPRIETARIO ( type: esriFieldTypeString, alias: Proprietário, length: 1000 )
- OBJECTID ( type: esriFieldTypeOID, alias: OBJECTID )
- UF ( type: esriFieldTypeString, alias: UF, length: 50 )
- CEG ( type: esriFieldTypeString, alias: CEG, length: 21, desc: Código criado para identificar única e inequivocamente o empreendimento de geração de energia elétrica. Uma vez criado, o acompanhará, independente de alteração de atributos ou proprietários da usina )
- Latitude( type: esriFieldTypeDouble)
- Longitude( type: esriFieldTypeDouble)
- Regiao ( type: esriFieldTypeString, alias: Região do Brasil)
- NM_MUNICIP ( type: esriFieldTypeString, alias: Município)

### Data Storage

In [33]:
import os
output_dir = 'Outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_file = os.path.join(output_dir, 'geodataframe.csv')
gdf.to_csv(output_file, index=False)